### TPC Benchmark Comparison  

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import config, tools, compare, bq_reservations

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999
pd.set_option('precision', 7)

#### Test Identification and Setup

In [ ]:
reserved_slots = None

benchmark = compare.QueryCompare()

# str, TPC test - either 'ds' or 'h'
benchmark.test     = 

# int, TPC scale factor (in GB)
benchmark.scale    = 1

# str, configuration identifying number i.e. '01', '02A', '03J'
benchmark.cid      = 

# list of str, systems to run benchmark test
benchmark.systems = #  ["sf"] or ["bq"] or ["bq", "sf"]

# int, TPC query stream number
benchmark.stream_n = 0

# str, description of test run (no _ characters)
# * comment one or the other out depending on system under test
# * add additional identifying info based on test goal
benchmark.desc      = f"{reserved_slots}-flex-slot-pc-cache-off"  # BigQuery
#benchmark.desc     = "on-demand-2XL-warehouse-cache-off"  # Snowflake 

# print general debug statements
benchmark.verbose       = True

# print query text executed
benchmark.verbose_query = False

# print status after each query
benchmark.verbose_iter  = True

# all the database system to cache and used cached results
benchmark.cache = False

# run qualifying data queries
benchmark.qual = False

# save data to file
benchmark.save = True

# auto timestamp generation
benchmark.set_timestamp_dir()

In [ ]:
print("Saving data to: ", benchmark.results_dir)

#### Slot Reservation

In [ ]:
if reserved_slots is not None:
    res_name = benchmark.data_source + "_" + benchmark.desc
    res_name = res_name.replace("_", "-").lower()
    print(f"Reservation Name: {res_name}")
    res = bq_reservations.BQR(fp_service_account_key=config.gcp_cred_file,
                              project=config.gcp_project,
                              location=config.gcp_location)
    res.assign_slots(slots=reserved_slots,
                     plan='FLEX',
                     name=res_name,
                     project=config.gcp_project)
    res.inventory()
    print("Commitments:")
    print(res.list_commitments)
    print("Reservations:")
    print(res.list_reservations)
    print("Assignments:")
    print(res.list_assignments)
    time.sleep(65)

#### Single Query

#### Query Stream Sequence

In [ ]:
# create the tpc query sequence for the stream
seq = tools.tpc_stream(test=benchmark.test, n=benchmark.stream_n)
print("Sequence:")
print(seq)

In [ ]:
%%time
# run an query stream sequence
benchmark.run(seq=seq)

#### Query Result Comparison

In [ ]:
results_dir = None
if len(benchmark.systems) > 1:
    df_results = benchmark.compare()
    #df_results
    
    x = df_results.equal.sum()/len(df_results)
    print(f"TPC Random Seed: {config.random_seed}")
    print("Query Stream Results Identical: {:2.2f}%".format(x*100))
    print("Non-Matching Query Results:",
          list(df_results.loc[df_results.equal == False, "q_sf"].values))
    df_results.loc[df_results.equal == False, ["q_bq", "equal_percent"]]
    results_dir = benchmark.results_dir

In [ ]:
%store results_dir

#### Delete slot commit, reservation and assignment

In [ ]:
if reserved_slots is not None:
    res.delete_all()
    res.inventory()
    print("Commitments:")
    print(res.list_commitments)
    print("Reservations:")
    print(res.list_reservations)
    print("Assignments:")
    print(res.list_assignments)

### Optional Audio Notification

In [ ]:
from IPython.display import Audio

In [ ]:
Audio('./data/2000_space_oddessy_completed.wav', autoplay=True)

In [ ]:
Audio('./data/George Gershwin - I Got Rhythm - Karstein Djupdal.mp3', autoplay=True)